# IRAF Tutorial

This time, you will learn very basic tasks of IRAF. This includes


* Print information of images (`IMSTAT`, `CCDLIST`, `IMHEADER`, `HSELECT`)
* Arithmetics on images: copying an image and adding, subtracting, multiplying, and dividing an image with others (`IMCOPY`, `IMARITH`) 
* Combining: make master bias, dark, and flat (`IMCOMBINE`)
* Preprocessing
* Primitive photometry (`IMEXAM`, `DAOEDIT`)
* Apperture photometry (`PHOT`)

I will explain the usage of IRAF/PyRAF in conda environment, so it may slightly differ from normal IRAF, but basically the same.

## Run PyRAF

You can start IRAF environment (*not* IRAF itself!) by typing

    source activate iraf

on the terminal. `iraf` is the name of conda environment you set when creating the environment, and you may have used `iraf27`, etc. Then you are now in the IRAF environment run on Python 2.7. At a suitable directory, type

    mkiraf

and answer the question with `xgterm`. Typing `ls` will show you `login.cl` and `uparms` directory. `login.cl` is the file which saves basic properties: It's a *settings* file in short. It will be used when you run IRAF **in this directory only**. You have to run the IRAF in the directory where you have `login.cl`. `uparms` is a directory which will save the changes you make to the default settings of IRAF.

Now, you can run IRAF by typing
    
    cl
    
or run PyRAF by typing

    pyraf
    
I recommend to use PyRAF, since AstroConda does not fully guarantee the stability of IRAF CL. The `IMEXAM`, for example, does not correctly work if you use `cl`. You will see the following on terminal with the prompt "`-->`":

<img src='../images/IRAF_01.png' width='500'>

## Display Image

PyRAF supports few LINUX commands, such as `ls` or `cd` without an exclamation mark (`!`). But to display image, you must use `!ds9` or `!ginga`. You should also use (`!`) when delete files using `rm`. 

Since IRAF is extremely comfortable when used with `SAO ds9`, I will use `ds9` throughout this note.

Display the image:

<img src='../images/IRAF_02.png' width='500'>

* **TIP**: It's better to append the ampersand (`&`) at the end of `ds9`. Try without it and see what happens.
* **TIP**: Always try to use `tab` key on your keyboard to "autocomplete"!


In the `HR30/` directory, you have four images and `bias/` and `flat/` directories. The four images are the real images. I first will make master bias and flat.

## Bias Combine

First, make the list of bias files. Go to the bias directory by "`--> cd bias`" and type

    --> ls *.fits > bias.list
    
This will make a file `bias.list` containing the list of all files that are bias. Then

    --> epar imcombine

`epar` means "edit parameters. The task `IMCOMBINE` will do task of combining images using many options. 

We will combine all 12 images in the `bias/` directory, and make one output named `bias.fits` using so-called **median combine**, while all other options left as default:

<img src='../images/IRAF_bias_combine.png' width='500'>

In IRAF commands, `@filename` means "open the file named `filename`, and put the contents here". Since `bias.list` contains the list of file names of bias images, 

    @bias.list == KCD095601.fits, KCD095602.fits, ....... , KCD095627.fits
    
See how efficient it is?

Clicking `Execute`, you will see that a new file appeared. Check it by 

    --> !ds9 -zscale bias.fits &

In this tutorial, we will neglect the dark, because it is extremely small compared to that of bias and almost constant over the image in modern CCDs. **So regard this master bias as "bias+dark".**

## Flat Combine

To make master flat, you have to subtract master bias from combined flat image. All are identical as that of bias, but subtract master bias from the final combined flat.

    --> cd ../flat/
    --> ls *.fits > flat.list
    --> epar imcombine

I want to make the combined image as `flat0.fits` first. Then save the result of subtracting bias, i.e., "`flat0 - bias`", as `flat.fits`. Do the median combine first:

<img src='../images/IRAF_flat_combine.png' width='500'>

Now copy the master bias to here and subtract it from the combined flat:

    --> cp ../bias/bias.fits .
    --> imarith flat0.fits - bias.fits flat.fits
    --> !ds9 -zscale flat.fits &
    
It's very intuitive: `IMARITH image1 operator image2 output` means do the operation (`operator` can be `+, -, *, /`) to images (`image1`, `image2`) and save the result as `output`. `IMARITH` of course means image arithmentics. You can open the full options window as you did for `IMCOMBINE` by typing

    --> epar imarith
    

### Note
There are some tasks which are made especially for bias, dark, and flat combine processes. They are called `ZEROCOMBINE`, `DARKCOMBINE`, and `FLATCOMBINE`. I usually avoid teaching these because it veils what's happening in IRAF, and thus not very good for educational purpose. If you want, you can freely try them. Manuals are kindly available from STScI: Just google with keywords such as "iraf zerocombine" and you will find the STScI website manuals.


## Preprocessing

Preprocessing is simply "(raw image - master bias)/(master flat)". 

    --> cd ..
    --> cp bias/bias.fits .
    --> cp flat/flat.fits .
    --> ls
    bias  bias.fits  flat  flat.fits  KCD095628.fits  KCD095629.fits  KCD095630.fits  KCD095631.fits

Conceptually what we have to do is first 

    imarith object_image - bias.fits output1
    
and then do 

    imarith output1 / flat.fits output2

Because we have 4 images, which is few, you can do this for each of image. However, there is cleverer way to do it.

    --> ls KCD*.fits > obj.list
    --> cp obj.list output1.list
    --> cp obj.list output2.list
    --> !perl -pi -e "s/KCD/b_KCD/g" output1.list
    --> !perl -pi -e "s/KCD/bf_KCD/g" output2.list
    --> imarith @obj.list - bias.fits @output1.list
    --> imarith @output1.list / flat.fits @output2.list
    --> ls
    bf_KCD095628.fits  bias              b_KCD095630.fits  KCD095628.fits  obj.list
    bf_KCD095629.fits  bias.fits	     b_KCD095631.fits  KCD095629.fits  output1.list
    bf_KCD095630.fits  b_KCD095628.fits  flat              KCD095630.fits  output2.list
    bf_KCD095631.fits  b_KCD095629.fits  flat.fits         KCD095631.fits

Here, `perl -pi -e` command is used for changing all the strings "`KCD`" to "`b_KCD`" or "`bf_KCD`":

    perl -pi -e "s/string1/string2/g" filename
                 ^ ^       ^          ^

First `s` means we will change the strings, **from "`string1`" to "`string2`"**, within the file `filename`. I used `b` for bias corrected, and `bf` for bias and flat corrected. You can display all the images

    --> !ds9 -zscale bf_*.fits -single &
    
* **TIP**: Hit `tab` key and the next image will be displayed. Can you find something? What do you think it is?

## File Information

You may want to see the brief information, such as statistics or the so-called "header information", of images. You can do this via many ways.


### Statistics from IRAF

There is a task named `IMSTATISTICS` which gives some statistics for each image:

    --> imstatistics *.fits
    #               IMAGE      NPIX      MEAN    STDDEV       MIN       MAX
           KCD095628.fits   4194304     5590.     2655.     4098.    65354.
           KCD095629.fits   4194304     5599.     2662.     4111.    65349.
           KCD095630.fits   4194304     5582.     2643.     4116.    65361.
           KCD095631.fits   4194304     5574.     2666.     4116.    65358.
         b_KCD095628.fits   4194304     1395.     2307.      -59.    61141.
         b_KCD095629.fits   4194304     1403.     2315.      -46.    61143.
         b_KCD095630.fits   4194304     1386.     2293.     -44.5    61145.
         b_KCD095631.fits   4194304     1378.     2319.      -43.    61141.
        bf_KCD095628.fits   4194304   0.07362    0.1266    -3.118     3.335
        bf_KCD095629.fits   4194304   0.07374    0.1271    -2.826     3.402
        bf_KCD095630.fits   4194304   0.07279    0.1263      -3.6     3.402
        bf_KCD095631.fits   4194304    0.0726    0.1273    -3.118       3.4
                bias.fits   4194304     4196.     1345.     4141.    65275.
                flat.fits   4194304    18973.     1960.       8.5    40058.

* **Question**: Why do you think the results `bf_*.fits` have such low min/max values? Do you have better idea to cope with this issue? This becomes problemetic for IRAF: see `DAOEDIT` below.
* **Question**: From file explorer, you can see the sizes of each image. The original `KCD*.fits` images are about 8.4 MB while the newly generated images are about twice larger. Why do you think this is so?


### Header from ds9
On `ds9`, you can click on `file -> header` to see all the header contents.

### Header from IRAF
You can use `IMHEADER`:

    --> imheader *.fits
    KCD095628.fits[2048,2048][ushort]: P/2006HR30
    KCD095629.fits[2048,2048][ushort]: P/2006HR30
    KCD095630.fits[2048,2048][ushort]: P/2006HR30
    KCD095631.fits[2048,2048][ushort]: P/2006HR30
    b_KCD095628.fits[2048,2048][real]: P/2006HR30
    b_KCD095629.fits[2048,2048][real]: P/2006HR30
    b_KCD095630.fits[2048,2048][real]: P/2006HR30
    b_KCD095631.fits[2048,2048][real]: P/2006HR30
    bf_KCD095628.fits[2048,2048][real]: P/2006HR30
    bf_KCD095629.fits[2048,2048][real]: P/2006HR30
    bf_KCD095630.fits[2048,2048][real]: P/2006HR30
    bf_KCD095631.fits[2048,2048][real]: P/2006HR30
    bias.fits[2048,2048][real]: bias
    flat.fits[2048,2048][real]: flat

The result is slightly different from IRAF, because of the Python nature. If you want to see the full header, you can do:

    --> imheader KCD095628.fits lo+

which means "long+". If you want to see only the exposure time, for example, which is `EXPTIME` in header keyword, you can use "`| grep`":
    
    --> imheader *.fits lo+ | grep 'EXPTIME'
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                120.0 / [sec] Total integration time
    EXPTIME =                  0.0 / [sec] Total integration time
    EXPTIME =                 10.0 / [sec] Total integration time

More flexibility comes with `HSELECT` which means header select:

    --> hselect *.fits "$I, EXPTIME, UT" yes
    KCD095628.fits	120.0	11:04:53
    KCD095629.fits	120.0	11:09:00
    KCD095630.fits	120.0	11:13:15
    KCD095631.fits	120.0	11:17:22
    b_KCD095628.fits	120.0	11:04:53
    b_KCD095629.fits	120.0	11:09:00
    b_KCD095630.fits	120.0	11:13:15
    b_KCD095631.fits	120.0	11:17:22
    bf_KCD095628.fits	120.0	11:04:53
    bf_KCD095629.fits	120.0	11:09:00
    bf_KCD095630.fits	120.0	11:13:15
    bf_KCD095631.fits	120.0	11:17:22
    bias.fits	0.0	09:30:25
    flat.fits	10.0	19:27:15

You may understand what this task gets and gives. `I` is for "inputs", and it prints the filenames.


### `CCDLIST`
There is another useful tool similar to `IMHEADER` If you have used `ZEROCOMBINE`, `DARKCOMBINE`,  and `FLATCOMBINE`. It will show you which preprocesses have been done for each image. But since we used `IMARITH` only, this result will be almost identical to that of `IMHEADER` and no more information will be there. To use `CCDLIST`, you have to activate some packages:

    --> imred
    imred/:
     argus/         ctioslit/       hydra/          kpnocoude/      vtel/
     bias/          dtoi/           iids/           kpnoslit/
     ccdred/        echelle/        irred/          quadred/
     crutil/        generic/        irs/            specred/
    --> ccdred
    ccdred/:
     badpiximage    ccdlist         combine         mkillumcor      setinstrument
     ccdgroups      ccdmask         darkcombine     mkillumflat     zerocombine
     ccdhedit       ccdproc         flatcombine     mkskycor
     ccdinstrument  ccdtest         mkfringecor     mkskyflat
    --> ccdlist *.fits
    KCD095628.fits[2048,2048][ushort][none][]:P/2006HR30
    KCD095629.fits[2048,2048][ushort][none][]:P/2006HR30
    KCD095630.fits[2048,2048][ushort][none][]:P/2006HR30
    KCD095631.fits[2048,2048][ushort][none][]:P/2006HR30
    b_KCD095628.fits[2048,2048][real][none][]:P/2006HR30
    b_KCD095629.fits[2048,2048][real][none][]:P/2006HR30
    b_KCD095630.fits[2048,2048][real][none][]:P/2006HR30
    b_KCD095631.fits[2048,2048][real][none][]:P/2006HR30
    bf_KCD095628.fits[2048,2048][real][none][]:P/2006HR30
    bf_KCD095629.fits[2048,2048][real][none][]:P/2006HR30
    bf_KCD095630.fits[2048,2048][real][none][]:P/2006HR30
    bf_KCD095631.fits[2048,2048][real][none][]:P/2006HR30
    bias.fits[2048,2048][real][none][]:bias
    flat.fits[2048,2048][real][none][]:flat


## Primitive Photometry

### `IMEXAMINE`
For photometry, let's use `IMEXAM` first. IRAF becomes extremely powerful when you use the interactive mode with the aid of `SAO ds9`. 

    --> !ds9 bf_KCD095628.fits &
    --> imexam

If you type `imexam`, your mouse cursor will blink. Put cursor on a star you are interested in, and hit `a` key. On terminal, you will see the information:

    #   COL    LINE     COORDINATES
    #     R    MAG    FLUX     SKY    PEAK    E   PA BETA ENCLOSED   MOFFAT DIRECT
    1034.81 1036.09 1034.81 1036.09
       8.36  23.77   3.093 0.06386  0.3435 0.21  -80 4.83     2.34     2.94   2.79

The meaning of the result is explained at STScI manual website ([here](http://stsdas.stsci.edu/cgi-bin/gethelp.cgi?imexamine)). The lines starting with `#` are comment lines, and they correspond to the next lines composed of numbers, which indicate the corresponding parameter. In the above, for example, the sky is estimated to be 0.05386 and the star flux is 3.093, with FWHM 2.94.

Hitting `r` will show you the radial profile of the star from the photocenter. `s` shows the surface plot. These may be useful if you do the psf photometry in the future using IRAF.

In the interactive mode, you can exit by hitting `q` key. Hit `?` key to see the manual or help on the terminal. You can scroll down with `enter` key and escape by hitting the `q` key. The manual here is a short version of the website version.


### `DAOEDIT`
In digiphot.daophot, there are many tasks. Quite many people do use the stand alone version ([see here](http://www.astrobetter.com/wiki/DAOphot)). Since we will not use IRAF intensively throughout this semester, I will just briefly mention the tasks.

